# Migración algoritmo optimización portafolios

## Carga librerias

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from datetime import date, timedelta
from IPython.display import clear_output
#Prueba

## Funciones de apoyo

In [ ]:
def get_time_delta(date: date , days: int, format: str = '%Y-%m-%d'):
    """this function returns a date that is days before the date passed as argument and its original day

    Args:
        date (date): date to be used as reference
        days (int): number of days for the time delta

    Returns:
        [str,str]: end date and start date in the format specified
    """
    return (date - timedelta(days=days)).strftime(format), date.strftime(format)

## Descarga datos historicos de los ultimos dos años

### Descarga de Cripto

In [ ]:

start_date,end_date = get_time_delta(date.today(), 730)

data = yf.download(['BTC-USD','ETH-USD','BNB-USD'], 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Adj Close"]]
data.reset_index(drop=True, inplace=True)
data["Date"] = data["Date"].dt.date
data["Date"] = pd.to_datetime(data["Date"])
fecha_minima=min(data["Date"])
data=data.set_index(data["Date"])
del data["Date"]
data = data.reorder_levels([1,0], axis=1)
data.columns = data.columns.droplevel()

In [98]:
data.to_csv('../data/Curated/Crypto_History.csv')

### Descarga de Acciones SP500

In [ ]:
#load data from SP500
sp500 = pd.read_csv('../data/SP500.csv')
symbols = sp500["Symbol"].tolist()

In [ ]:
data = yf.download(symbols, 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Adj Close"]]
data.reset_index(drop=True, inplace=True)
data["Date"] = data["Date"].dt.date
data["Date"] = pd.to_datetime(data["Date"])
fecha_minima=min(data["Date"])
data=data.set_index(data["Date"])
del data["Date"]
data = data.reorder_levels([1,0], axis=1)
data.columns = data.columns.droplevel()

In [ ]:
data.to_csv('../data/Curated/SP500_History.csv')

## Calculo retornos diarios y mensuales

In [ ]:
retornos_diarios = data['Adj Close'].pct_change()
retornos_mensuales = data['Adj Close'].resample('M').ffill().pct_change()

In [ ]:
retornos_diarios=retornos_diarios.drop(index=fecha_minima)
rendimientos_promedio=retornos_diarios.mean()

## Calculo retornos promedios de los activos

In [ ]:
rendimientos_promedio